In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cuda:1"

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
)

In [4]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Talklife MoC

In [5]:
%run load_talklifemoc.py

In [6]:
%run load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([18604, 384])

Baseline: LSTM classification (window=20)

In [8]:
num_epochs = 100
hidden_dim_sizes = [100, 200, 300, 384]
num_layers = 1
bidirectional = True
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [9]:
# create indices for kfold
fold_col_names = [c for c in df.columns if "fold" in c]
fold_list = []
for foldc in fold_col_names:
    fold_list.append(
        (
            df[df[foldc] == "train"].index,
            df[df[foldc] == "dev"].index,
            df[df[foldc] == "test"].index,
        )
    )
fold_list = tuple(fold_list)

In [10]:
size = 20
(
    bilstm_history_20,
    best_bilstm_history_20,
    _,
    __,
) = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=None,
    split_indices=fold_list,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/lstm_history_20_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/lstm_history_20_focal_2_kfold_best_model.csv


In [11]:
bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_158673/1802967379.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
100        0.1          0.0001         0.811653  0.504240   0.518992   
                        0.0005         0.813875  0.523137   0.530253   
                        0.0010         0.818372  0.536526   0.542656   
           0.2          0.0001         0.812639  0.503141   0.518730   
                        0.0005         0.814269  0.522828   0.531625   
                        0.0010         0.819268  0.536402   0.543293   
200        0.1          0.0001         0.813302  0.497192   0.511880   
                        0.0005         0.812406  0.525450   0.527271   
                        0.0010         0.812603  0.533006   0.532568   
           0.2          0.0001         0.813481  0.498118   0.513166   
                        0.0005         0.811188  0.526187   0.527315   
                        0.0010         0.814413  0.536920   0.538093   
300        0.1          0.0001         0.806529  0.495275   0.511194   
                        0.0005         0.820988  0.528162   0.542958   
                        0.0010         0.812263  0.534341   0.534704   
           0.2          0.0001         0.808106  0.495008   0.511881   
                        0.0005         0.819447  0.524285   0.538356   
                        0.0010         0.816706  0.533641   0.537195   
384        0.1          0.0001         0.812639  0.490415   0.516912   
                        0.0005         0.818767  0.529626   0.538013   
                        0.0010         0.810507  0.533165   0.528840   
           0.2          0.0001         0.810202  0.490449   0.513692   
                        0.0005         0.818032  0.529616   0.537464   
                        0.0010         0.808446  0.532644   0.526355   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
100        0.1          0.0001         0.503343        0.805385  0.506174   
                        0.0005         0.523667        0.809540  0.529253   
                        0.0010         0.537171        0.811777  0.535854   
           0.2          0.0001         0.501783        0.805747  0.503892   
                        0.0005         0.524581        0.808664  0.524767   
                        0.0010         0.536064        0.812193  0.534395   
200        0.1          0.0001         0.494726        0.807997  0.497223   
                        0.0005         0.528161        0.806747  0.531878   
                        0.0010         0.540165        0.807469  0.538102   
           0.2          0.0001         0.495585        0.808331  0.495499   
                        0.0005         0.530688        0.804496  0.528915   
                        0.0010         0.542410        0.807456  0.537136   
300        0.1          0.0001         0.498440        0.804038  0.503465   
                        0.0005         0.524309        0.814180  0.528162   
                        0.0010         0.542475        0.806858  0.539087   
           0.2          0.0001         0.496110        0.804343  0.498665   
                        0.0005         0.520998        0.812985  0.525997   
                        0.0010         0.535984        0.810999  0.538820   
384        0.1          0.0001         0.486599        0.807219  0.493979   
                        0.0005         0.527312        0.811874  0.528784   
                        0.0010         0.541945        0.805177  0.538916   
           0.2          0.0001         0.489095        0.803454  0.493540   
                        0.0005         0.528650        0.811276  0.530617   
                        0.0010         0.544633        0.803246  0.538914   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
100    

In [12]:
best_bilstm_history_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.823425,0.535746,"[0.8985516432668844, 0.470326409495549, 0.2383...",0.542492,"[0.8944412896712727, 0.46893491124260356, 0.26...",0.530540,"[0.9026999490575649, 0.47172619047619047, 0.21...",None,0.818640,...,True,300,0.1,0.001,1,focal,2,True,5,64
0,None,0.809450,0.532076,"[0.8885941644562334, 0.47839569981393426, 0.22...",0.534870,"[0.9030243261012492, 0.41013824884792627, 0.29...",0.545814,"[0.8746179317371371, 0.5739087301587301, 0.188...",None,0.802718,...,True,300,0.1,0.001,12,focal,2,True,5,64
0,None,0.803913,0.535218,"[0.885061944606603, 0.4672897196261682, 0.2533...",0.526763,"[0.9018506278916061, 0.4086181277860327, 0.269...",0.551070,"[0.8688869077941925, 0.5456349206349206, 0.238...",None,0.799216,...,True,300,0.1,0.001,123,focal,2,True,5,64


In [13]:
best_bilstm_history_20[
    [
        "f1",
        "f1_scores",
        "precision",
        "recall",
        "valid_f1",
        "valid_f1_scores",
        "valid_precision",
        "valid_recall",
        "hidden_dim",
        "dropout_rate",
        "learning_rate",
        "seed",
        "loss_function",
        "k_fold",
        "batch_size",
    ]
]

,f1,f1_scores,precision,recall,valid_f1,valid_f1_scores,valid_precision,valid_recall,hidden_dim,dropout_rate,learning_rate,seed,loss_function,k_fold,batch_size
0,0.535746,"[0.8985516432668844, 0.470326409495549, 0.2383...",0.542492,0.530540,0.547567,"[0.894514977187066, 0.472851492268968, 0.27533...",0.553971,0.546272,300,0.1,0.001,1,focal,True,64
0,0.532076,"[0.8885941644562334, 0.47839569981393426, 0.22...",0.534870,0.545814,0.534559,"[0.8833741881681029, 0.47677827495756825, 0.24...",0.540213,0.551690,300,0.1,0.001,12,focal,True,64
0,0.535218,"[0.885061944606603, 0.4672897196261682, 0.2533...",0.526763,0.551070,0.535082,"[0.8817487223599656, 0.4677265500794912, 0.255...",0.528678,0.553072,300,0.1,0.001,123,focal,True,64


In [14]:
best_bilstm_history_20["f1"].mean()

0.5343467031879768

In [15]:
best_bilstm_history_20["precision"].mean()

0.5347084507917814

In [16]:
best_bilstm_history_20["recall"].mean()

0.542474556661026

In [17]:
np.stack(best_bilstm_history_20["f1_scores"]).mean(axis=0)

array([0.89073592, 0.47200394, 0.24030025])

In [18]:
np.stack(best_bilstm_history_20["precision_scores"]).mean(axis=0)

array([0.89977208, 0.42923043, 0.27512284])

In [19]:
np.stack(best_bilstm_history_20["recall_scores"]).mean(axis=0)

array([0.88206826, 0.53042328, 0.21493213])